In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import random
import shutil
from tqdm.auto import tqdm
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

# Load the dataset

In [ ]:
IMAGE_FOLDER = "tusimple_processed/images"
MASK_FOLDER = "tusimple_processed/masks"

In [ ]:
def create_dirs_if_not_exist(path):
    if not os.path.exists(path):
        os.makedirs(path)
        print("Directory created:", path)
    else:
        print("Directory already exists:", path)

In [ ]:
create_dirs_if_not_exist(IMAGE_FOLDER)
create_dirs_if_not_exist(MASK_FOLDER)

### Copy Images to the Image Directory

In [ ]:
CLIPS_PATH = "/kaggle/input/tusimple/TUSimple/train_set/clips"

# iterate through each directory
for clip_dir in os.listdir(CLIPS_PATH):
    clip_dir_path = os.path.join(CLIPS_PATH, clip_dir)

    print("Processing Clip:", clip_dir)
    # iterate through each sub directory
    for frame_dir in os.listdir(clip_dir_path):
        frame_path = os.path.join(clip_dir_path, frame_dir, "20.jpg")

        # check if file is present
        if not os.path.isfile(frame_path):
            continue

        # create new filename based on last 2 directory names
        tmp = frame_path[:-7].split('/')[-2:]
        new_filename = f"{tmp[0]}_{tmp[1]}.jpg"
        new_file_path = os.path.join(IMAGE_FOLDER, new_filename)

        # copy the file
        shutil.copy(frame_path, new_file_path)

In [ ]:
print("Total images in dataset:", len(os.listdir(IMAGE_FOLDER)))

### Create Masks for the Images

In [ ]:
# load the dataset json files
df1 = pd.read_json("/kaggle/input/tusimple/TUSimple/train_set/label_data_0313.json", lines = True)
df2 = pd.read_json("/kaggle/input/tusimple/TUSimple/train_set/label_data_0531.json", lines = True)
df3 = pd.read_json("/kaggle/input/tusimple/TUSimple/train_set/label_data_0601.json", lines = True)

# combine into single dataframe
df = pd.concat([df1, df2, df3])
df.head()

In [ ]:
# create function for generating masks
def generate_lane_mask(row):
    # create a mask of all zeros
    mask = np.zeros((728, 1280, 1), dtype=np.uint8)

    # extract data from the row
    h_samples = row.h_samples
    lanes = row.lanes
    raw_file = row.raw_file

    # create mask: lane = 1, non-lane = 0
    for lane in lanes:
        # exclude -2 datapoints
        h_samples_filtered = [y for x, y in zip(lane, h_samples) if x != -2]
        lane_filtered = [x for x in lane if x != -2]

        # create array of lane points
        lane_points = np.array(list(zip(lane_filtered, h_samples_filtered)))

        # update lane mask
        cv2.polylines(mask, [lane_points], isClosed = False, color = (255, 255, 255), thickness = 15)
    
    # generate mask filename
    temp = raw_file[:-7].split('/')[-2:]
    mask_filename = f"{temp[0]}_{temp[1]}.jpg"
    mask_filename_path = os.path.join(MASK_FOLDER, mask_filename)

    # write the mask image
    cv2.imwrite(mask_filename_path, mask)

In [ ]:
# generate masks
for index, row in tqdm(df.iterrows(), total=len(df)):
    generate_lane_mask(row)

In [ ]:
print("Total masks in dataset:", len(os.listdir(MASK_FOLDER)))

# Data Visualization

In [ ]:
def visualize_image(image_name):
    # get paths
    image_path = os.path.join(IMAGE_FOLDER, image_name)
    mask_path = os.path.join(MASK_FOLDER, image_name)
    
    # read the image and mask
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(mask_path)
    
    # plot the image
    plt.figure(figsize = (20, 8))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Road Image")

    plt.subplot(1, 2, 2)
    plt.imshow(mask, cmap="gray")
    plt.title("Ground Truth Mask")
    
    # show the images
    plt.show()

In [ ]:
for i in range(5):
    image_name = random.choice(os.listdir(IMAGE_FOLDER))
    visualize_image(image_name)

# Process Data for Training and Testing

In [ ]:
train_image_folder = "tusimple_processed/train/images"
test_image_folder = "tusimple_processed/test/images"
train_mask_folder = "tusimple_processed/train/masks"
test_mask_folder = "tusimple_processed/test/masks"

In [ ]:
for folder_path in [train_image_folder, test_image_folder, train_mask_folder, test_mask_folder]:
    create_dirs_if_not_exist(folder_path)

In [ ]:
from sklearn.model_selection import train_test_split
images = [file for file in os.listdir(IMAGE_FOLDER) if file.endswith(".jpg")]
masks = [file for file in os.listdir(MASK_FOLDER) if file.endswith(".jpg")]

# split for train and test
train_images, test_images = train_test_split(images, test_size=0.1, random_state=42)

In [ ]:
len(train_images), len(test_images)

In [ ]:
# move files to the corresponding folders
for file in train_images:
    # move the image and mask
    source = os.path.join(IMAGE_FOLDER, file)
    destination = os.path.join(train_image_folder, file)
    shutil.move(source, destination)
    
    source = os.path.join(MASK_FOLDER, file)
    destination = os.path.join(train_mask_folder, file)
    shutil.move(source, destination)

for file in test_images:
    # move the image and mask
    source = os.path.join(IMAGE_FOLDER, file)
    destination = os.path.join(test_image_folder, file)
    shutil.move(source, destination)
    
    source = os.path.join(MASK_FOLDER, file)
    destination = os.path.join(test_mask_folder, file)
    shutil.move(source, destination)

## Feature Engineering

In [ ]:
import tensorflow as tf

def load_image(image_path, mask_path):
    size = [224, 224]
    
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, size)
    image = image / 255.0  

    
    kernel = tf.constant([[0., -1., 0.],
                          [-1., 5., -1.],
                          [0., -1., 0.]], dtype=tf.float32)
    kernel = tf.reshape(kernel, [3, 3, 1, 1])

    channels = tf.split(image, num_or_size_splits=3, axis=-1)
    sharpened_channels = []
    for c in channels:
        c_sharp = tf.nn.conv2d(tf.expand_dims(c, axis=0), kernel, strides=1, padding="SAME")
        sharpened_channels.append(tf.squeeze(c_sharp, axis=0))
    image = tf.concat(sharpened_channels, axis=-1)
    image = tf.clip_by_value(image, 0.0, 1.0)  

  
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_jpeg(mask, channels=1)
    mask = tf.image.resize(mask, size)
    mask = mask / 255.0  
    
    return image, mask


In [ ]:
def dataset_from_folder(image_folder, mask_folder):
    image_files = sorted([os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(".jpg")])
    mask_files = sorted([os.path.join(mask_folder, file) for file in os.listdir(mask_folder) if file.endswith(".jpg")])

    dataset = tf.data.Dataset.from_tensor_slices((image_files, mask_files))
    dataset = dataset.map(lambda image_path, mask_path: load_image(image_path, mask_path))
    
    return dataset

In [ ]:
# load dataset from folder
train_dataset = dataset_from_folder(train_image_folder, train_mask_folder)
test_dataset = dataset_from_folder(test_image_folder, test_mask_folder)

In [ ]:
# set config
BATCH_SIZE = 16
BUFFER_SIZE = 1000

# optimize for performance improvement
train_dataset = train_dataset.cache().shuffle(BUFFER_SIZE).repeat().batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE)
test_datset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
def display_sample(image_list):
    plt.figure(figsize=(10, 10))

    titles = ["Image", "True Mask", "Predicted Mask"]

    for i in range(len(image_list)):
        plt.subplot(1, len(image_list), i+1)
        plt.title(titles[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(image_list[i]))
        plt.axis("off")

    plt.show()

In [ ]:
for image, mask in train_dataset.take(1):
    display_sample([image[0], mask[0]])

# Model Creation - VGG-UNet

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Dropout, Activation, Add, BatchNormalization, Conv2D, Concatenate, UpSampling2D
from tensorflow.keras.models import Model

def VGG16_UNet(input_shape=(None, None, 3)):
    '''U net architechture with VGG16 encoder'''
    def Conv2DReluBatchNorm(n_filters, kernel_size, stride, inputs):
        x = Conv2D(n_filters, (1, 1), strides=1, 
                   padding="same", kernel_initializer='glorot_normal', activation="elu")(inputs)
        x = Conv2D(n_filters, kernel_size, strides=stride, 
                   padding="same", kernel_initializer='glorot_normal', activation="elu")(x)
        x = BatchNormalization()(x)
        x = Dropout(rate=0.25)(x, training=True) # traininig + test-time ropout!
        return x
        
    # VGG16 encoder
    vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
    
    # Unfreeze VGG16 layers
    for layer in vgg16.layers:
        layer.trainable = True
    
    # Encoder layers
    inputs = vgg16.input
    layer1 = vgg16.get_layer("block1_conv2").output
    layer2 = vgg16.get_layer("block2_conv2").output
    layer3 = vgg16.get_layer("block3_conv3").output
    layer4 = vgg16.get_layer("block4_conv3").output
    layer5 = vgg16.get_layer("block5_conv3").output

    # Decoder layers
    merge6 = Concatenate(axis=-1)([UpSampling2D(size=(2, 2))(layer5), layer4])
    layer6 = Conv2DReluBatchNorm(512, (3, 3), (1, 1), merge6)

    merge7 = Concatenate(axis=-1)([UpSampling2D(size=(2, 2))(layer6), layer3])
    layer7 = Conv2DReluBatchNorm(256, (3, 3), (1, 1), merge7)

    merge8 = Concatenate(axis=-1)([UpSampling2D(size=(2, 2))(layer7), layer2])
    layer8 = Conv2DReluBatchNorm(512, (3, 3), (1, 1), merge8)

    merge9 = Concatenate(axis=-1)([UpSampling2D(size=(2, 2))(layer8), layer1])
    layer9 = Conv2DReluBatchNorm(512, (3, 3), (1, 1), merge9)

    output = Conv2D(1, (1, 1), strides=(1, 1), activation="sigmoid", name="output")(layer9)

    return Model(inputs=inputs, outputs=output)

model = VGG16_UNet(input_shape=(224, 224, 3)) # Adjust input shape as needed

In [ ]:
import tensorflow
from tensorflow.keras import backend as K

def dice_coefficent(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    mu = y_pred[:, :, :, 0]
    y_pred_f = K.flatten(mu)
    intersection = K.sum(y_true_f * y_pred_f)
    smooth = 1.0
    return (2 * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficent(y_true, y_pred)

def recall_smooth(y_true, y_pred):
    y_pred_f = K.flatten(y_pred)
    y_true_f = K.flatten(y_true)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection / (K.sum(y_true_f) + K.epsilon()))

def precision_smooth(y_true, y_pred):
    y_pred_f = K.flatten(y_pred)
    y_true_f = K.flatten(y_true)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection / (K.sum(y_true_f) + K.epsilon()))

def accuracy(y_true, y_pred):
    y_pred_f = K.flatten(y_pred)
    y_true_f = K.flatten(y_true)

    # True positives
    true_positives = K.sum(K.round(K.clip(y_true_f * y_pred_f, 0, 1)))
    
    # True negatives
    true_negatives = K.sum(K.round(K.clip((1- y_true_f) * (1 - y_pred_f), 0, 1)))
    
    # Total pixels
    total_pixels = K.cast(tensorflow.size(y_true_f), K.floatx())
    
    # Accuracy
    accuracy_value = (true_positives + true_negatives) / total_pixels

    return accuracy_value

In [ ]:
# compile the model
model.compile(optimizer="adam", loss=dice_loss, metrics=[dice_coefficent, precision_smooth, recall_smooth, accuracy])
print(f'Number of parameters: {model.count_params()}')

In [ ]:
# plot the model
tf.keras.utils.plot_model(model, show_shapes=True, expand_nested=False, dpi=64)

In [ ]:
# create mask from prediction
def create_mask(pred_mask):
    # round to closest
    pred_mask = tf.math.round(pred_mask)
    return pred_mask

def show_predictions(dataset=None, num=1):
    if dataset:
        for images, masks in dataset.take(num):
            pred_mask = model.predict(images)
            pred_mask = create_mask(pred_mask[0])
            display_sample([images[0], masks[0], pred_mask])

In [ ]:
show_predictions(train_dataset, 1)

In [ ]:
# callbacks and logs
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, Callback
import datetime

class DisplayCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        show_predictions(test_dataset, 1)
        print(f"Sample Prediction after epoch {epoch}\n")

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

callbacks = [
    DisplayCallback(), 
    TensorBoard(logdir, histogram_freq=-1),
    EarlyStopping(patience=5, verbose=1),
    ModelCheckpoint("best_model.keras", verbose=1, save_best_only=True)
]

In [ ]:
EPOCHS = 10
steps_per_epoch = len(os.listdir(train_image_folder)) // BATCH_SIZE
validation_steps = len(os.listdir(test_image_folder)) // BATCH_SIZE

# Train the Model

In [ ]:
# Train the model
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

## Plot the Metrics

In [ ]:
# plot train and val accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(["Train", "Test"], loc="upper left")

# plot train and val loss
plt.subplot(1, 2, 2)
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(["Train", "Test"], loc="upper left")

plt.tight_layout()
plt.savefig("training_history.svg")
plt.show()

## Load the Best Model

In [ ]:
best_model = tf.keras.models.load_model(
    "best_model.keras",
    custom_objects={
        "dice_loss": dice_loss,
        "dice_coefficent": dice_coefficent,
        "precision_smooth": precision_smooth,
        "recall_smooth": recall_smooth,
        "accuracy": accuracy
    }
)
best_model.summary()

## Test Predictions

In [ ]:
from tensorflow.keras.preprocessing.image import array_to_img

def save_prediction_plot(image_list, index=0, save_dir="prediction_plots"):
    os.makedirs(save_dir, exist_ok=True)
    titles = ["Image", "True Mask", "Predicted Mask"]

    plt.figure(figsize=(10, 10))
    for i in range(len(image_list)):
        plt.subplot(1, len(image_list), i + 1)
        plt.title(titles[i])
        plt.imshow(array_to_img(image_list[i]))
        plt.axis("off")

    filepath = os.path.join(save_dir, f"prediction_{index}.png")
    plt.savefig(filepath)
    plt.close()
    print(f"✅ Prediction plot saved: {filepath}")

def show_and_save_predictions(dataset=None, num=1, save_dir="prediction_plots"):
    if dataset:
        os.makedirs(save_dir, exist_ok=True)
        for i, (images, masks) in enumerate(dataset.take(num)):
            pred_mask = model.predict(images)
            pred_mask = create_mask(pred_mask[0])
            display_sample([images[0], masks[0], pred_mask])
            save_prediction_plot([images[0], masks[0], pred_mask], index=i, save_dir=save_dir)

show_and_save_predictions(test_dataset, 10)

## Test from Image Path

In [ ]:
def load_test_image(image_path):
    size = [224, 224]
    
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, size)
    image = image / 255.0 # normalize to [0, 1]

    kernel = tf.constant([[0., -1., 0.],
                          [-1., 5., -1.],
                          [0., -1., 0.]], dtype=tf.float32)
    kernel = tf.reshape(kernel, [3, 3, 1, 1])

    channels = tf.split(image, num_or_size_splits=3, axis=-1)
    sharpened_channels = []
    for c in channels:
        c_sharp = tf.nn.conv2d(tf.expand_dims(c, axis=0), kernel, strides=1, padding="SAME")
        sharpened_channels.append(tf.squeeze(c_sharp, axis=0))
    image = tf.concat(sharpened_channels, axis=-1)
    image = tf.clip_by_value(image, 0.0, 1.0)  
    
    return image

In [ ]:
def display_test_sample(image_list):
    plt.figure(figsize=(10, 10))

    titles = ["Image", "Predicted Mask"]

    for i in range(len(image_list)):
        plt.subplot(1, len(image_list), i+1)
        plt.title(titles[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(image_list[i]))
        plt.axis("off")

    plt.show()

In [ ]:
image_path = "/kaggle/input/tusimple/TUSimple/train_set/clips/0313-1/10000/20.jpg"
image = load_test_image(image_path)
test_image = tf.expand_dims(image, axis=0)
pred_mask = model.predict([test_image])
pred_mask = create_mask(pred_mask[0])

display_test_sample([image, pred_mask])

# Convert Image Sequence to Video Using Opencv

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

In [ ]:
def load_and_preprocess_frame(image_path):
    size = [224, 224]
    
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, size)
    image = image / 255.0  # normalize to [0, 1]

    kernel = tf.constant([[0., -1., 0.],
                          [-1., 5., -1.],
                          [0., -1., 0.]], dtype=tf.float32)
    kernel = tf.reshape(kernel, [3, 3, 1, 1])

    channels = tf.split(image, num_or_size_splits=3, axis=-1)
    sharpened_channels = []
    for c in channels:
        c_sharp = tf.nn.conv2d(tf.expand_dims(c, axis=0), kernel, strides=1, padding="SAME")
        sharpened_channels.append(tf.squeeze(c_sharp, axis=0))
    image = tf.concat(sharpened_channels, axis=-1)
    image = tf.clip_by_value(image, 0.0, 1.0)  
    
    # Add batch dimension
    image = tf.expand_dims(image, axis=0)
    
    return image

In [ ]:
def create_mask(pred_mask):
    pred_mask = tf.math.round(pred_mask)
    return pred_mask

In [ ]:
def postprocess_mask(pred_mask, original_frame):
    pred_mask = pred_mask.numpy()
    pred_mask = pred_mask[0, :, :, 0] if pred_mask.ndim == 4 else pred_mask[0]
    pred_mask = (pred_mask * 255).astype(np.uint8)
    pred_mask = cv2.resize(pred_mask, (original_frame.shape[1], original_frame.shape[0]))

    # Create a blank image for the colored mask
    mask_colored = np.zeros_like(original_frame)

    # Yellow lane color (BGR): (0, 255, 255)
    mask_colored[pred_mask > 0] = [0, 255, 255]

    # Blend the original frame with the mask
    blended = cv2.addWeighted(original_frame, 1.0, mask_colored, 0.6, 0)

    return blended

In [ ]:
from tensorflow.keras.models import load_model
import os

# Input and output paths
path = "/kaggle/input/tusimple/TUSimple/test_set/clips/0530/1492626265087865031_0"
out_path = "imgToVid"
out_video_name = "driving_car_lanes.mp4"
out_video_full_path = os.path.join(out_path, out_video_name)

# Create output directory if it doesn't exist
os.makedirs(out_path, exist_ok=True)

# Load the Keras lane detection model (replace with your model path)
model = load_model('best_model.keras', custom_objects={
        "dice_loss": dice_loss,
        "dice_coefficent": dice_coefficent,
        "precision_smooth": precision_smooth,
        "recall_smooth": recall_smooth,
        "accuracy": accuracy
    })  # Update with your model path

# Get list of images and sort them
pre_imgs = [f for f in os.listdir(path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
pre_imgs.sort()

# Full paths to images
img_paths = [os.path.join(path, i) for i in pre_imgs]

# Read the first image to get size
frame = cv2.imread(img_paths[0])
if frame is None:
    raise ValueError(f"Failed to read first image: {img_paths[0]}")

height, width = frame.shape[:2]
cv2_fourcc = cv2.VideoWriter_fourcc(*"mp4v")
fps = 12
video = cv2.VideoWriter(out_video_full_path, cv2_fourcc, fps, (width, height))

# Process each frame
for img_path in img_paths:
    frame = cv2.imread(img_path)
    if frame is None:
        print(f"Warning: Could not read image {img_path}")
        continue
    
    input_frame = load_and_preprocess_frame(img_path)
    
    pred_mask = model.predict(input_frame, verbose=0)
    pred_mask = create_mask(pred_mask)
    
    output_frame = postprocess_mask(pred_mask, frame)
    
    video.write(output_frame)

video.release()
print("Video with masked lane predictions saved to:", out_video_full_path)

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def play(filename):
    html = ''
    video = open(filename,'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html += '<video width=1000 controls autoplay loop><source src="%s" type="video/mp4"></video>' % src 
    return HTML(html)

play(out_video_full_path)

# Packaging Model, Training Metrics, and Output Video into a ZIP File

In [ ]:
import csv

with open("training_history.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write header
    writer.writerow(["Epoch", "Train_Loss", "Train_Accuracy", "Val_Loss", "Val_Accuracy"])
    
    # Write data
    for epoch in range(len(history.history['loss'])):
        writer.writerow([
            epoch + 1,
            history.history['loss'][epoch],
            history.history['accuracy'][epoch],
            history.history['val_loss'][epoch],
            history.history['val_accuracy'][epoch]
        ])

In [ ]:
import zipfile

import os

files = [
    "training_history.csv",
    "training_history.svg",
    "best_model.keras",
    "imgToVid/driving_car_lanes.mp4"
]

plots_dir = "prediction_plots"

zip_filename = "model_and_metrics.zip"

def add_file_to_zip(file_name, zipf):
    zipf.write(file_name)
    print(f"✅ Added to zip: {file_name}")

def add_folder_to_zip(directory, zipf):
    for foldername, subfolders, filenames in os.walk(directory):
        for filename in filenames:
            filepath = os.path.join(foldername, filename)
           
            arcname = os.path.relpath(filepath, start=os.path.dirname(directory))
            zipf.write(filepath, arcname=arcname)
        print(f"✅ Added to zip: {foldername}")

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in files:
        add_file_to_zip(file, zipf)

    add_folder_to_zip(plots_dir, zipf)

print(f"✅ Zip created: {zip_filename} (includes model, metrics CSV, video, and prediction plots)")

In [ ]:
from IPython.display import FileLink, display, HTML

# Create and display download link
display(FileLink(zip_filename))

display(HTML("<h3 style='color:red;'>⚠️ Don't forget to click and download the zip file before closing the session!</h3>"))